### Import libraries

In [1]:
import sys
sys.path.append("../")

In [2]:
import os
from os import environ
import numpy as np
from random import choices
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
import skimage.io as io

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.transforms as mtrans
%matplotlib inline

In [5]:
from preprocess.common import load_nii

In [6]:
from torch.utils.data import DataLoader
from torchcontrib.optim import SWA

# ---- My utils ----
from utils.data_augmentation import data_augmentation_selector
from utils.dataload import *
from utils.training import *

In [7]:
train_aug, train_aug_img, val_aug = data_augmentation_selector("none", 224, 224)

Using None Data Augmentation


In [8]:
data_partition = "train"
general_aug, img_aug = train_aug, train_aug_img
normalization = "standardize"
fold_system = "patient"
label_type = "mask"


train_dataset = MMsDataset(
    mode=data_partition, transform=general_aug, img_transform=img_aug,
    folding_system=fold_system, normalization=normalization, label_type=label_type
)

In [9]:
data_partition = "validation"
general_aug, img_aug = train_aug, train_aug_img
normalization = "standardize"
fold_system = "patient"
label_type = "mask"


validation_dataset = MMsDataset(
    mode=data_partition, transform=general_aug, img_transform=img_aug,
    folding_system=fold_system, normalization=normalization, label_type=label_type
)

In [24]:
data_partition = "train"
general_aug, img_aug = train_aug, train_aug_img
normalization = "standardize"
fold_system = "exclusion"
label_type = "mask"


train_dataset_weakly = MMsWeaklyDataset(
    mode=data_partition, transform=general_aug, img_transform=img_aug,
    folding_system=fold_system, normalization=normalization, label_type=label_type,
    exclusion_patients=validation_dataset.df["External code"].unique()
)

In [20]:
len(validation_dataset.df["External code"].unique())

30

In [23]:
len(train_dataset_weakly.df["External code"].unique())

175

In [11]:
val_patients = validation_dataset.df["External code"].unique()

In [21]:
print(np.intersect1d(val_patients, train_dataset.df["External code"].unique()))
print(np.intersect1d(val_patients, train_dataset_weakly.df["External code"].unique()))

[]
[]


In [20]:
train_loader = DataLoader(dataset, batch_size=2, shuffle=True, drop_last=False)

In [28]:
for indx, (image, label) in enumerate(train_loader):
    break

comeon
comeon


In [9]:
data_partition = "validation"
general_aug, img_aug = train_aug, train_aug_img
normalization = "standardize"
fold_system = "patient"
label_type = "vendor_label_full"
data_fold = 0


val_dataset = MMsDataset(
    mode=data_partition, transform=general_aug, img_transform=img_aug,
    folding_system=fold_system, normalization=normalization, label_type=label_type,
    train_fold=data_fold, val_fold=None,
)

---------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------

In [10]:
train_dataset.df.head()

NameError: name 'train_dataset' is not defined

In [11]:
val_dataset.df.head()

,External code,Vendor,Centre,Labeled,Type,Phase,Slice
0,A3B7E5,B,2,True,ED,29,0
1,A3B7E5,B,2,True,ED,29,1
2,A3B7E5,B,2,True,ED,29,2
3,A3B7E5,B,2,True,ED,29,3
4,A3B7E5,B,2,True,ED,29,4


#### Value counts

In [12]:
train_dataset.df.Vendor.value_counts()

A    1428
B    1220
C     436
Name: Vendor, dtype: int64

In [13]:
val_dataset.df.Vendor.value_counts()

B    326
A    310
C    138
Name: Vendor, dtype: int64

#### Patient intersection

In [14]:
np.intersect1d(train_dataset.df["External code"].unique(), val_dataset.df["External code"].unique())

array([], dtype=object)